#Summarize Dialogue assignment

Using Flan-T5 model and knkarthick/dailogsum dataset from hugging face

**Goal**: Compare Human provided summary (Baseline) with Model generated summary of the dialogue

In [2]:
!pip install --upgrade pip
!pip install --disable-pip-version-check \
      torch==1.13.1 \
      torchdata==0.5.1 --quiet

!pip install transformers==4.27.2 \
      datasets==2.11.0 --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 9.6 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 855.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.3.0+cu121 requires torch==2.3.0, but you have torch 1.13.1 which is incompatible.
torchtext 0.18.0 requires torc

In [3]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer, GenerationConfig

##Load dataset and view samples

In [4]:
dataset = load_dataset("knkarthick/dialogsum")
len(dataset)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/knkarthick___csv/knkarthick--dialogsum-cd36827d3490488d/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

3

In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 12460
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 1500
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 500
    })
})

In [6]:
#Print couple of dialogues from the dataset to analyze it

example_indices = [10, 1400]

dash_line = "-".join('' for i in range(100))

for i, index in enumerate(example_indices):
  print(dash_line)
  print(f"Example {i+1}")
  print(dash_line)
  print(f"INPUT DIALOGUE: Index:{index}")
  print(dataset['test'][index]['dialogue'])
  print(dash_line)
  print("BASELINE HUMAN SUMMARY:")
  print(dataset['test'][index]['summary'])
  print(dash_line)
  print()

---------------------------------------------------------------------------------------------------
Example 1
---------------------------------------------------------------------------------------------------
INPUT DIALOGUE: Index:10
#Person1#: Happy Birthday, this is for you, Brian.
#Person2#: I'm so happy you remember, please come in and enjoy the party. Everyone's here, I'm sure you have a good time.
#Person1#: Brian, may I have a pleasure to have a dance with you?
#Person2#: Ok.
#Person1#: This is really wonderful party.
#Person2#: Yes, you are always popular with everyone. and you look very pretty today.
#Person1#: Thanks, that's very kind of you to say. I hope my necklace goes with my dress, and they both make me look good I feel.
#Person2#: You look great, you are absolutely glowing.
#Person1#: Thanks, this is a fine party. We should have a drink together to celebrate your birthday
-------------------------------------------------------------------------------------------------

##1 : Summarize dialouge without prompt engineering


Load FlanT5 base model and tokenizer

In [7]:
model_name = 'google/flan-T5-base'

model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [8]:
#Test the tokenizer
sample = "This is a beautiful day"
sentense_encoded = tokenizer(sample, return_tensors='pt')
print(sentense_encoded)
print(sentense_encoded['input_ids'][0])

{'input_ids': tensor([[100,  19,   3,   9, 786, 239,   1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]])}
tensor([100,  19,   3,   9, 786, 239,   1])


In [9]:
#Decode encoded sentense
sentense_decoded = tokenizer.decode(sentense_encoded['input_ids'][0])
print(sentense_decoded)

#Decode encoded sentense without special tokens
sentense_decoded = tokenizer.decode(sentense_encoded['input_ids'][0],
                                    skip_special_tokens=True)
print(sentense_decoded)

This is a beautiful day</s>
This is a beautiful day



Explore how the Flan-T5-base model works on the example dialogues without prompt engineering

In [10]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 12460
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 1500
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 500
    })
})


In [11]:
dialogue = dataset['test'][0]['dialogue']
summary = dataset['test'][0]['summary']

#Generate input embeddings
embeddings = tokenizer(dialogue, return_tensors='pt')

#Generate response from the model
response_embeddings = model.generate(embeddings['input_ids'],
                          max_new_tokens=50)

print(response_embeddings[0])

tensor([    0,  1713,   345, 13515,   536,  4663,    10,   283,     7,     5,
        31676,     6,    27,   174,    25,    12,   240,     3,     9,     3,
        12194,   257,    21,   140,     5,     1])


In [12]:
def display_output(index, dialogue, summary, response):
  print(dash_line)
  print(f"Example {index}")
  print(dialogue)
  print(dash_line)
  print("Human baseline summary:")
  print(dash_line)
  print(summary)
  print(dash_line)
  print("Generated Response:")
  print(dash_line)
  print(response)

In [13]:
for i, index in enumerate(example_indices):
  dialogue = dataset['test'][index]['dialogue']
  summary = dataset['test'][index]['summary']

  #Generate input embeddings
  embeddings = tokenizer(dialogue, return_tensors='pt')

  #Generate response from the model
  response_embeddings = model.generate(embeddings['input_ids'],
                            max_new_tokens=50)

  #Translate response embeddings
  response = tokenizer.decode(response_embeddings[0],
                              skip_special_tokens=True)

  display_output(i+1, dialogue, summary, response)

---------------------------------------------------------------------------------------------------
Example 1
#Person1#: Happy Birthday, this is for you, Brian.
#Person2#: I'm so happy you remember, please come in and enjoy the party. Everyone's here, I'm sure you have a good time.
#Person1#: Brian, may I have a pleasure to have a dance with you?
#Person2#: Ok.
#Person1#: This is really wonderful party.
#Person2#: Yes, you are always popular with everyone. and you look very pretty today.
#Person1#: Thanks, that's very kind of you to say. I hope my necklace goes with my dress, and they both make me look good I feel.
#Person2#: You look great, you are absolutely glowing.
#Person1#: Thanks, this is a fine party. We should have a drink together to celebrate your birthday
---------------------------------------------------------------------------------------------------
Human baseline summary:
--------------------------------------------------------------------------------------------------

## 2 . In Context Learning : Summarize dialouge with an instruction prompt


In [16]:
def display_output_for_prompting_egs(index, prompt, summary, prompt_type, response):
  print(dash_line)
  print(f"Example {index}")
  print(f"Input Prompt:\n{prompt}")
  print(dash_line)
  print(f"Human baseline summary:\n{summary}")
  print(dash_line)
  print(f"Generated Response: {prompt_type}\n")
  print(response)

### 1 : Zero Shot Prompting

In [17]:
for i, index in enumerate(example_indices):
  dialogue = dataset['test'][index]['dialogue']
  summary = dataset['test'][index]['summary']

  prompt = f"""
Summarize the dialogue.

  {dialogue}

Summary:
  """

  input = tokenizer(prompt, return_tensors='pt')

  output = model.generate(input['input_ids'],
                        max_new_tokens=50)

  text = tokenizer.decode(output[0],
                          skip_special_tokens=True)

  display_output_for_prompting_egs(i+1, prompt, summary, "ZERO SHOT :1: ", response)

---------------------------------------------------------------------------------------------------
Example 1
Input Prompt:

Summarize the dialogue. 

  #Person1#: Happy Birthday, this is for you, Brian.
#Person2#: I'm so happy you remember, please come in and enjoy the party. Everyone's here, I'm sure you have a good time.
#Person1#: Brian, may I have a pleasure to have a dance with you?
#Person2#: Ok.
#Person1#: This is really wonderful party.
#Person2#: Yes, you are always popular with everyone. and you look very pretty today.
#Person1#: Thanks, that's very kind of you to say. I hope my necklace goes with my dress, and they both make me look good I feel.
#Person2#: You look great, you are absolutely glowing.
#Person1#: Thanks, this is a fine party. We should have a drink together to celebrate your birthday
  
Summary:
  
---------------------------------------------------------------------------------------------------
Human baseline summary:
#Person1# attends Brian's birthday party

####ZERO SHOT Prompting : With Different prompt

In [18]:
for i, index in enumerate(example_indices):
  dialogue = dataset['test'][index]['dialogue']
  summary = dataset['test'][index]['summary']

  prompt = f"""
Dialogue:

  {dialogue}

What is going on?
  """

  input = tokenizer(prompt, return_tensors='pt')

  output_ids = model.generate(input['input_ids'],
                              max_new_tokens=50)
  output_text = tokenizer.decode(output_ids[0],
                                 skip_special_tokens=True)

  display_output_for_prompting_egs(i+1, prompt, summary, "ZERO SHOT", response)

---------------------------------------------------------------------------------------------------
Example 1
Input Prompt:

Dialogue:

  #Person1#: Happy Birthday, this is for you, Brian.
#Person2#: I'm so happy you remember, please come in and enjoy the party. Everyone's here, I'm sure you have a good time.
#Person1#: Brian, may I have a pleasure to have a dance with you?
#Person2#: Ok.
#Person1#: This is really wonderful party.
#Person2#: Yes, you are always popular with everyone. and you look very pretty today.
#Person1#: Thanks, that's very kind of you to say. I hope my necklace goes with my dress, and they both make me look good I feel.
#Person2#: You look great, you are absolutely glowing.
#Person1#: Thanks, this is a fine party. We should have a drink together to celebrate your birthday
  
What is going on?
  
---------------------------------------------------------------------------------------------------
Human baseline summary:
#Person1# attends Brian's birthday party. Bria

### 2 . ONE SHOT Prompt Engineering

In [19]:
def make_prompt(reference_input, question_input):
  #print(f"called make_prompt")
  prompt = ""
  for i in range(len(reference_input)):
    input_dialogue = reference_input[i]['dialogue']
    input_summary = reference_input[i]['summary']

    #print(f"{i} : {input_summary}")

    prompt += f"""

  Dialogue:

  {input_dialogue}

  What is going on?
  {input_summary}

    """

  question_dialogue = question_input['dialogue']

  prompt += f"""
Dialogue:

{question_dialogue}

What is going on?
"""

  return prompt

In [20]:
for i, index in enumerate(example_indices):
  print(i, index)

0 10
1 1400


In [21]:
reference_input = [dataset['test'][0]]

for i, index in enumerate(example_indices):
  question_input = dataset['test'][index]
  human_summary = dataset['test'][index]['summary']

  prompt = make_prompt(reference_input,  question_input)

  input = tokenizer(prompt, return_tensors='pt')
  output_ids = model.generate(input['input_ids'],
                              max_new_tokens=50)
  output_text =tokenizer.decode(output_ids[0],
                                skip_special_tokens=True)

  display_output_for_prompting_egs(i+1, prompt, human_summary, "ONE SHOT", output_text)

Token indices sequence length is longer than the specified maximum sequence length for this model (600 > 512). Running this sequence through the model will result in indexing errors


---------------------------------------------------------------------------------------------------
Example 1
Input Prompt:

  
  Dialogue:

  #Person1#: Ms. Dawson, I need you to take a dictation for me.
#Person2#: Yes, sir...
#Person1#: This should go out as an intra-office memorandum to all employees by this afternoon. Are you ready?
#Person2#: Yes, sir. Go ahead.
#Person1#: Attention all staff... Effective immediately, all office communications are restricted to email correspondence and official memos. The use of Instant Message programs by employees during working hours is strictly prohibited.
#Person2#: Sir, does this apply to intra-office communications only? Or will it also restrict external communications?
#Person1#: It should apply to all communications, not only in this office between employees, but also any outside communications.
#Person2#: But sir, many employees use Instant Messaging to communicate with their clients.
#Person1#: They will just have to change their commun

### 3 . FEW SHOT Prompt Engineering

In [22]:
reference_input = []
reference_eg_indices = [0, 100, 150, 120, 7]
for i, index in enumerate(reference_eg_indices):
  reference_input += [dataset['test'][index]]

for i, index in enumerate(example_indices):
  question_input = dataset['test'][index]
  human_summary = dataset['test'][index]['summary']

  prompt = make_prompt(reference_input,  question_input)

  input = tokenizer(prompt, return_tensors = 'pt')
  output_ids = model.generate(input['input_ids'],
                               max_new_tokens=50)
  output_text =tokenizer.decode(output_ids[0],
                                skip_special_tokens=True)

  display_output_for_prompting_egs(i+1, prompt, human_summary, "FEW SHOT", output_text)

---------------------------------------------------------------------------------------------------
Example 1
Input Prompt:

  
  Dialogue:

  #Person1#: Ms. Dawson, I need you to take a dictation for me.
#Person2#: Yes, sir...
#Person1#: This should go out as an intra-office memorandum to all employees by this afternoon. Are you ready?
#Person2#: Yes, sir. Go ahead.
#Person1#: Attention all staff... Effective immediately, all office communications are restricted to email correspondence and official memos. The use of Instant Message programs by employees during working hours is strictly prohibited.
#Person2#: Sir, does this apply to intra-office communications only? Or will it also restrict external communications?
#Person1#: It should apply to all communications, not only in this office between employees, but also any outside communications.
#Person2#: But sir, many employees use Instant Messaging to communicate with their clients.
#Person1#: They will just have to change their commun

## 3 . Generative Confiugration Parameters for Inference

In [23]:
generation_config = GenerationConfig(max_new_tokens = 50,
                                     do_sample = True,
                                     Temperature = 0.8)

input = tokenizer(prompt, return_tensors = 'pt')
output_ids = model.generate(input['input_ids'],
                            generation_config = generation_config)
output_text = tokenizer.decode(output_ids[0],
                               skip_special_tokens=True)

display_output_for_prompting_egs(1, prompt, human_summary, "FEW SHOT", output_text)

---------------------------------------------------------------------------------------------------
Example 1
Input Prompt:

  
  Dialogue:

  #Person1#: Ms. Dawson, I need you to take a dictation for me.
#Person2#: Yes, sir...
#Person1#: This should go out as an intra-office memorandum to all employees by this afternoon. Are you ready?
#Person2#: Yes, sir. Go ahead.
#Person1#: Attention all staff... Effective immediately, all office communications are restricted to email correspondence and official memos. The use of Instant Message programs by employees during working hours is strictly prohibited.
#Person2#: Sir, does this apply to intra-office communications only? Or will it also restrict external communications?
#Person1#: It should apply to all communications, not only in this office between employees, but also any outside communications.
#Person2#: But sir, many employees use Instant Messaging to communicate with their clients.
#Person1#: They will just have to change their commun